# 	Text Prediction | Long Short-Term Memory 

### Scikit-Learn, Tensorflow, Keras, Python 

In [1]:
%matplotlib inline
import urllib # to read the data directly from url
from urllib import request
import re
import io
import numpy as np
import pandas as pd
import keras
import cv2
import numpy
import sys



from keras.layers import LSTM
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from tensorflow.keras import backend
from tensorflow.python.framework import ops
from keras.layers import Dropout, MaxPooling2D, Conv2D, Dense
from keras.datasets import cifar10
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
#from skimage.color import rgb2grey

from keras.models import Sequential



Using TensorFlow backend.


##  Generative Models for Text


#### Building a generative model to mimic the writing style of prominent British Mathematician, Philosopher, prolific writer, and political activist, Bertrand Russell.

#### Data Source : Project Gutenberg http://www.gutenberg.org/ebooks/author/355 in text format





####  LSTM to train the model to predict Russell's writings


In [9]:
open('all_Russells.txt','w').close()
##function to extract data from the respective urls directly and save to a file
def extractText(textUrl):
    data = urllib.request.urlopen(textUrl).read().decode('utf-8').replace('\r', ' ').replace('\n', ' ').replace("\'", ' ').replace("'\ufeff", ' ').replace("'\u03b1'", ' ')
    removed_end = data.split('END OF THIS PROJECT GUTENBERG')[0]
    removed_start = removed_end.split("START OF THIS PROJECT GUTENBERG EBOOK")[1]
    return(str(removed_start.encode("utf-8")))


all_text = open('all_Russells.txt', 'a')
phylosophy = extractText('http://www.gutenberg.org/cache/epub/5827/pg5827.txt')
analysis = extractText('http://www.gutenberg.org/cache/epub/2529/pg2529.txt')
knowledge = extractText('http://www.gutenberg.org/files/37090/37090-0.txt')
all_text.write(phylosophy)
all_text.write(analysis)
all_text.write(knowledge)
all_text.close()




#### Character-level representation for this model by using extended ASCII that has N = 256 characters.
#### Each character will be encoded into a an integer using its ASCII code. Rescale the integers to the range [0; 1], because LSTM uses a sigmoid activation function. LSTM will receive the rescaled integers as its input.

In [10]:
all_Russells_data = open('all_Russells.txt').read()
all_Russells_data = all_Russells_data.lower()

sorted_list_set = sorted(list(set(all_Russells_data)))
int_char = dict((x, p) for p, x in enumerate(sorted_list_set))
sorted_list_set_number = len(all_Russells_data)
count_sorted_list_set = len(sorted_list_set)

#### Window size, e.g., W = 100.
#### Inputs to the network will be the first W −1 = 99 characters of each sequence, and the output of the network will be the Wth character of the sequence.Training the network to predict each character using the 99 characters that precede it. Slide the window in strides of S = 1 on the text. For example, if W = 5 and S = 1 and we want to train the network with the sequence ABRACADABRA, The first input to the network will be ABRA and the corresponding output will be C. The second input will be BRAC and the second output will be A, etc.

In [11]:
strider_window = 100
X = []
Y = []
for i in range(0, sorted_list_set_number - strider_window, 1):
    
    seq_in = all_Russells_data[i:i + strider_window]
    #print("seq_in {} strider_window {} ".format(seq_in,strider_window))
    seq_out = all_Russells_data[i + strider_window]
    #print("seq_out {} ,strider_window {}  ".format(seq_in,strider_window))
    X.append([int_char[char] for char in seq_in])
    Y.append(int_char[seq_out])
    


#### Output has to be encoded using a one-hot encoding scheme with N = 256 (or less) elements. This means that the network reads integers, but outputs a vector of N = 256 (or less) elements.

In [171]:
matched_text_count = len(X)
#print("matched {}  ".format(matched_text_count))

y = np_utils.to_categorical(Y)
#print("y {}  ".format(y))
X = numpy.reshape(X, (matched_text_count, strider_window, 1))
X = X / float(count_sorted_list_set)



#### Using a single hidden layer for the LSTM with N = 256 (or less) memory units.
#### Using Softmax output layer to yield a probability prediction for each of the characters between 0 and 1. This is actually a character classification problem with N classes. Choose log loss (cross entropy) as the objective function for the network (research what it means).

In [16]:
import warnings
warnings.simplefilter("ignore")

lstm_model = Sequential()
lstm_model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(y.shape[1], activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam')
location="epoch_loss-{epoch:02d}-{loss:.4f}.hdf5"
model_checkpoint = ModelCheckpoint(location, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [model_checkpoint]

lstm_model.fit(X, y, epochs=10, batch_size=1000, callbacks=callbacks_list)



Epoch 1/10
1209637/1209637 [==============================] - 2407s 2ms/step - loss: 2.8233
Epoch 2/10
1209637/1209637 [==============================] - 1868s 2ms/step - loss: 2.6640
Epoch 3/10
1209637/1209637 [==============================] - 1806s 1ms/step - loss: 2.5752
Epoch 4/10
1209637/1209637 [==============================] - 2372s 2ms/step - loss: 2.4836
Epoch 5/10
1209637/1209637 [==============================] - 2064s 2ms/step - loss: 2.3891
Epoch 6/10
1209637/1209637 [==============================] - 2584s 2ms/step - loss: 2.3012
Epoch 7/10
1209637/1209637 [==============================] - 3183s 3ms/step - loss: 2.2359
Epoch 8/10
1209637/1209637 [==============================] - 1825s 2ms/step - loss: 2.1835
Epoch 9/10
1209637/1209637 [==============================] - 1824s 2ms/step - loss: 2.1393
Epoch 10/10
1209637/1209637 [==============================] - 1828s 2ms/step - loss: 2.0994


####  We do not use a test dataset. We are using the whole training dataset to learn the probability of each character in a sequence. We are not seeking for a very accurate model. Instead we are interested in a generalization of the dataset that can mimic the gist of the text.
#### Choosing a reasonable number of epochs4 for training, considering the computational power (e.g., 30, although the network will need more epochs to yield a better model).
####  Using  model checkpointing to keep the network weights to determine each time an improvement in loss is observed at the end of the epoch. Finding the best set of weights in terms of loss.
#### Using the network with the best weights to generate 1000 characters, using the following text as initialization of the network: There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object.


In [179]:

# load the network weights
filename = "epoch_loss-10-2.0994.hdf5"
lstm_model.load_weights(filename)
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam')
sorted_list_int = dict((i, c) for i, c in enumerate(sorted_list_set))


p = "There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object.".lower()
matched_text = [int_char[char] for char in p]
matched_text = matched_text[0:100]
print("\"", ''.join([sorted_list_int[p] for p in matched_text]), "\"")
for i in range(1000):
    x = numpy.reshape(matched_text, (1, len(matched_text), 1))
    x = x / float(count_sorted_list_set)
    pred = lstm_model.predict(x, verbose=0)
    index = numpy.argmax(pred)
    res = sorted_list_int[index]
    sys.stdout.write(res)
    matched_text.append(index)
    matched_text = matched_text[1:len(matched_text)]


" there are those who take mental phenomena naively, just as they would physical phenomena. this schoo "
  th the that the semse of the soetere ooreoem of the soetere ohreop of  the saee to the poososition of the soetere ohreop of the soetere oore of  the seesed that the same thing  and the soetere ohreop of the soetere  ant aalinten                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         